In [32]:
import torch
import cv2

# 모델 로드
model = torch.load('yolov5s.pt')
model.eval()  # 모델을 evaluation 모드로 설정

# 웹캠 초기화
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    # Inference
    results = model([frame])  # 이미지를 모델에 통과시킴
    rendered_img = results.render()[0]  # 렌더링된 이미지를 가져옴

    cv2.imshow('YOLOv5', rendered_img)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


FileNotFoundError: [Errno 2] No such file or directory: 'yolov5s.pt'